# What's in an Avocado Toast: A Supply Chain Analysis

![](avocado_wallpaper.jpeg)

You find yourself in London, crafting a delectable avocado toast, a dish that has risen dramatically in popularity on breakfast menus since the 2010s. This straightforward recipe requires just five ingredients: a ripe avocado, half a lemon, a generous pinch of salt flakes, two slices of sourdough bread, and a good drizzle of extra virgin olive oil. Most of these ingredients are now staples in grocery stores, and as you will find with this project, that is no small feat!

In this project, you'll conduct a supply chain analysis of three ingredients used in avocado toast using the Open Food Facts database. This database contains extensive, openly-sourced information on various foods, including their origins. Through this analysis, you will gain an in-depth understanding of the complex supply chain involved in producing a single dish.

Three pairs of files are provided in the data folder:
- A CSV file for each ingredient, such as `avocado.csv`, with data about each food item and countries of origin.
- A TXT file for each ingredient, such as `relevant_avocado_categories`, containing only the category tags of interest for that food.

Here are some other key points about these files:
- Some of the rows of data in each of the three CSV files do not contain relevant data for your investigation. In each dataset, you will need to filter out rows with irrelevant data, based on values in the `categories_tags` column. Examples of categories are fruits, vegetables, and fruit-based oils. Filter the DataFrame to include only rows where `categories_tags` contains one of the tags in the relevant categories for that ingredient.
- Each row of data usually has multiple category tags in the `categories_tags` column.
There is a column in each CSV file called `origins_tags`, which contains strings for the country of origin of each item.

After completing this project, you'll be armed with a list of ingredients and their countries of origin and be well-positioned to launch into other analyses that explore how long, on average, these ingredients spend at sea.

[Open Food Facts database](https://world.openfoodfacts.org/)

In [73]:
import pandas as pd
import os
print(os.listdir())

['avocado_wallpaper.jpeg', 'data', 'notebook.ipynb']


In [74]:
avocado_df = pd.read_csv("data/avocado.csv", sep='\t')
avocado_df.head()

,code,lc,product_name_de,product_name_el,product_name_en,product_name_es,product_name_fi,product_name_fr,product_name_id,product_name_it,product_name_lt,product_name_lv,product_name_nb,product_name_nl,product_name_pl,product_name_ro,product_name_sl,product_name_sv,generic_name_de,generic_name_en,generic_name_es,generic_name_fr,generic_name_sv,quantity,serving_size,packaging,packaging_tags,brands,brands_tags,brand_owner,categories,categories_tags,labels,labels_tags,countries,countries_tags,stores,stores_tags,obsolete,obsolete_since_date,...,phosphorus_unit,iron_value,iron_unit,magnesium_value,magnesium_unit,zinc_value,zinc_unit,copper_value,copper_unit,manganese_value,manganese_unit,selenium_value,selenium_unit,fruits-vegetables-nuts_value,fruits-vegetables-nuts_unit,phylloquinone_value,phylloquinone_unit,link,off:food_groups,off:food_groups_tags,off:nova_groups,off:nova_groups_tags,off:nutriscore_grade,off:nutriscore_score,off:ecoscore_grade,off:ecoscore_score,off:ecoscore_data.missing_key_data,off:ecoscore_data.agribalyse.code,off:ecoscore_data.adjustments.origins_of_ingredients.value,off:ecoscore_data.adjustments.packaging.value,off:ecoscore_data.adjustments.packaging.non_recyclable_and_non_biodegradable_materials,off:ecoscore_data.adjustments.production_system.value,off:ecoscore_data.adjustments.threatened_species.value,sources_fields:org-database-usda:available_date,sources_fields:org-database-usda:fdc_category,sources_fields:org-database-usda:fdc_data_source,sources_fields:org-database-usda:fdc_id,sources_fields:org-database-usda:modified_date,sources_fields:org-database-usda:publication_date,data_sources
0,0059749979702,fr,NaN,NaN,NaN,NaN,NaN,Naturalia Avocado Oil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Naturalia,naturalia,NaN,"Aliments et boissons à base de végétaux, Alime...","en:plant-based-foods-and-beverages,en:plant-ba...",NaN,NaN,Canada,en:canada,NaN,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:fats,"en:fats-and-sauces,en:fats",2.0,en:2-processed-culinary-ingredients,b,1.0,unknown,NaN,1.0,NaN,-5.0,-15.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"App - yuka, Apps"
1,7610095131409,en,NaN,NaN,NaN,NaN,NaN,Avocado Bowl chips,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zweifel,zweifel,NaN,"Snacks, Salty snacks, Appetizers, Chips and fr...","en:snacks,en:salty-snacks,en:appetizers,en:chi...","Vegetarian, Vegan","en:vegetarian,en:vegan","Switzerland, World","en:switzerland,en:world",NaN,NaN,1,2023/03/21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:appetizers,"en:salty-snacks,en:appetizers",NaN,unknown,c,8.0,unknown,NaN,1.0,NaN,-5.0,-15.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"App - Yuka, Apps, Producers, Producer - zweifel"
2,4005514005578,en,NaN,NaN,Gelbe Linse Avocado Brotaufstrich,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tartex,tartex,NaN,de:Abendbrotsufstrich,de:abendbrotsufstrich,"Organic, EU Organic, EG-Öko-Verordnung","en:organic,en:eu-organic,en:eg-oko-verordnung",Germany,en:germany,NaN,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,b,0.0,unknown,NaN,1.0,NaN,-5.0,-15.0,1.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"App - yuka, Apps, App - smoothie-openfoodfacts"
3,0879890002513,en,NaN,NaN,Avocado toast chili lime,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States,en:united-states,NaN,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,en:4-ultra-processed-food-and-drink-products,NaN,NaN,unknown,NaN,1.0,NaN,-5.0,-15.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"App - Yuka, Apps, App - InFood"
4,0223086613685,en,NaN,NaN,Avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States,en:united-stat

In [75]:
avocado_df = avocado_df[['code', 'lc', 'product_name_en', 'quantity', 'serving_size', 'packaging_tags', 'brands', 'brands_tags', 'categories_tags', 'labels_tags', 'countries', 'countries_tags', 'origins', 'origins_tags']]
avocado_df.head()

,code,lc,product_name_en,quantity,serving_size,packaging_tags,brands,brands_tags,categories_tags,labels_tags,countries,countries_tags,origins,origins_tags
0,0059749979702,fr,NaN,NaN,NaN,NaN,Naturalia,naturalia,"en:plant-based-foods-and-beverages,en:plant-ba...",NaN,Canada,en:canada,NaN,NaN
1,7610095131409,en,NaN,NaN,NaN,NaN,Zweifel,zweifel,"en:snacks,en:salty-snacks,en:appetizers,en:chi...","en:vegetarian,en:vegan","Switzerland, World","en:switzerland,en:world",NaN,NaN
2,4005514005578,en,Gelbe Linse Avocado Brotaufstrich,NaN,NaN,NaN,Tartex,tartex,de:abendbrotsufstrich,"en:organic,en:eu-organic,en:eg-oko-verordnung",Germany,en:germany,NaN,NaN
3,0879890002513,en,Avocado toast chili lime,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States,en:united-states,NaN,NaN
4,0223086613685,en,Avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States,en:united-states,NaN,NaN


In [76]:
avocado_tags_open = open("data/relevant_avocado_categories.txt", "r")
avocado_tags = avocado_tags_open.read()
avocado_tags = avocado_tags.split()
print(avocado_tags)
print(type(avocado_tags))

['en:avocadoes', 'en:avocados', 'en:fresh-foods', 'en:fresh-vegetables', 'en:fruchte', 'en:fruits', 'en:raw-green-avocados', 'en:tropical-fruits', 'en:tropische-fruchte', 'en:vegetables-based-foods', 'fr:hass-avocados']
<class 'list'>


In [77]:
filtr = avocado_df["categories_tags"].str.split(",").dropna().apply(lambda x: any([b for b in x if b in avocado_tags]))

In [78]:
avocado_df = avocado_df.dropna(subset='categories_tags')

In [79]:
avocado_df = avocado_df[filtr]
print(avocado_df.shape)
avocado_df.head()

(182, 14)


,code,lc,product_name_en,quantity,serving_size,packaging_tags,brands,brands_tags,categories_tags,labels_tags,countries,countries_tags,origins,origins_tags
5,3662994002063,fr,NaN,3 fruits,NaN,NaN,la compagnie des fruits mûrs,la-compagnie-des-fruits-murs,"en:plant-based-foods-and-beverages,en:plant-ba...",NaN,France,en:france,NaN,NaN
6,8437013031011,fr,NaN,1 kg,NaN,NaN,NaN,NaN,"en:plant-based-foods-and-beverages,en:plant-ba...",NaN,France,en:france,NaN,NaN
14,4016249238155,de,NaN,135g,100g,de:gläschen,Allos,allos,"en:plant-based-foods-and-beverages,en:plant-ba...","en:organic,en:vegetarian,en:eu-organic,en:no-g...",Deutschland,en:germany,Europäische Union,en:european-union
17,8718963381532,de,NaN,NaN,NaN,NaN,NaN,NaN,"en:plant-based-foods-and-beverages,en:plant-ba...",NaN,Deutschland,en:germany,NaN,NaN
23,8436002746707,es,NaN,NaN,NaN,NaN,NaN,NaN,"en:plant-based-foods-and-beverages,en:plant-ba...",NaN,España,en:spain,NaN,NaN


In [80]:
avocado_UK = avocado_df[avocado_df["countries"] == "United Kingdom"]
avocado_UK

,code,lc,product_name_en,quantity,serving_size,packaging_tags,brands,brands_tags,categories_tags,labels_tags,countries,countries_tags,origins,origins_tags
361,00985833,en,Avacado,650 g,NaN,NaN,Marks & Spencer,marks-spencer,"en:plant-based-foods-and-beverages,en:plant-ba...",NaN,United Kingdom,en:united-kingdom,Peru,en:peru
381,00040464,en,Avocado,NaN,NaN,NaN,NaN,NaN,"en:plant-based-foods-and-beverages,en:plant-ba...",NaN,United Kingdom,en:united-kingdom,NaN,NaN
414,4088600100173,en,Avocado,100 g,NaN,en:mixed-plastic-unknown,Aldi,aldi,"en:plant-based-foods-and-beverages,en:plant-ba...",NaN,United Kingdom,en:united-kingdom,NaN,NaN
468,01307351,en,Avacados organic,NaN,NaN,"en:card-tray,en:ldpe-bag",Sainsbury’s SO organic,sainsbury-s-so-organic,"en:plant-based-foods-and-beverages,en:plant-ba...","en:organic,en:eu-organic,en:non-eu-agriculture...",United Kingdom,en:united-kingdom,NaN,NaN
508,5057172125395,en,Just Essentials Avocados,4pack,NaN,en:mixed-plastic-film-packet-to-recycle,Asda,asda,"en:plant-based-foods-and-beverages,en:plant-ba...","en:class-i,en:contains-stones",United Kingdom,en:united-kingdom,Peru,en:peru
510,23066755,en,Ready to Eat Avocados,2,NaN,en:mixed-plastic-bag,Asda,asda,"en:plant-based-foods-and-beverages,en:plant-ba...",NaN,United Kingdom,en:united-kingdom,"Spain, Peru","en:spain,en:peru"
708,03201985,en,Avocado,2,80g,"en:mixed-plastic-packet,en:plastic-film",Tesco,tesco,"en:plant-based-foods-and-beverages,en:plant-ba...","en:tesco-nurture,en:vitamin-e-source",United Kingdom,en:united-kingdom,"Chile, Peru","en:chile,en:peru"
781,10096369,en,Avocado,NaN,100 g,NaN,Tesco,tesco,"en:plant-based-foods-and-beverages,en:plant-ba...",NaN,United Kingdom,en:united-kingdom,NaN,NaN
850,00184915,en,Rich & creamy avocados,NaN,NaN,"en:card-tray,en:mixed-plastic-sleeve",By Sainsbury's,by-sainsbury-s,"en:plant-based-foods-and-beverages,en:plant-ba...",NaN,United Kingdom,en:united-kingdom,NaN,NaN
1190,01600322,en,Ripe & ready medium avocados,2,NaN,"en:card-tray,en:ldpe-bag",Sainsbury's,sainsbury-s,"en:plant-based-foods-and-beverages,en:plant-ba...",en:class-i,United Kingdom,en:united-kingdom,Israel,en:israel


In [81]:
avocado_UK["origins"].str.split().explode().str.replace(",", "").value_counts()

Peru      4
Spain     1
Chile     1
Israel    1
Name: origins, dtype: int64

In [82]:
top_avocado_origin = avocado_UK["origins"].str.split().explode().str.replace(",", "").value_counts().index[0]
top_avocado_origin

'Peru'

In [83]:
def top_origin(ingredient, country = "United Kingdom"):
    #wczytanie pliku csv
    ing_df = pd.read_csv("data/" + ingredient + ".csv", sep='\t')
    ing_df = ing_df[['code', 'lc', 'product_name_en', 'quantity', 'serving_size', 'packaging_tags', 'brands',               'brands_tags', 'categories_tags', 'labels_tags', 'countries', 'countries_tags', 'origins', 'origins_tags']]
    
    #wczytanie listy tagów z pliku txt
    ing_tags_open = open("data/relevant_" + ingredient + "_categories.txt", "r")
    ing_tags = ing_tags_open.read()
    ing_tags = ing_tags.split()
    
    #filtrowanie danych po tagach
    filtr = ing_df["categories_tags"].str.split(",").dropna().apply(lambda x: any([b for b in x if b in ing_tags]))
    ing_df = ing_df.dropna(subset='categories_tags')
    ing_df = ing_df[filtr]
    
    #filtrowanie danych po kraju, do którego dostarczono produkt
    ing_country = ing_df[ing_df["countries"] == country]
    
    #zliczanie krajów pochodzenia
    top_ingredient_origin = ing_country["origins"].str.split(",").explode().str.replace(",", "").value_counts().index[0]
    return top_ingredient_origin

In [84]:
top_olive_oil_origin = top_origin("olive_oil")
top_olive_oil_origin

'Greece'

In [85]:
top_sourdough_origin = top_origin("sourdough")
top_sourdough_origin

'United Kingdom'

In [86]:
top_avocado_origin = top_origin("avocado")
top_avocado_origin

'Peru'